![dvd_image](dvd_image.jpg)

Uma locadora de DVD precisa da sua ajuda! Eles querem descobrir por quantos dias um cliente alugará um DVD com base em alguns recursos e pediram ajuda a você. Eles querem que você experimente alguns modelos de regressão que ajudarão a prever o número de dias que um cliente alugará um DVD. A empresa deseja um modelo que produza um MSE de 3 ou menos em um conjunto de testes. O modelo que você criar ajudará a empresa a se tornar um planejamento de estoque mais eficiente.

In [1]:
# Importing necessary libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# Loading the data and checking it's format
rentals = pd.read_csv("rental_info.csv")

rentals.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [3]:
rentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [4]:
# Convert date to an coherent type 
rentals["rental_date"] = pd.to_datetime(rentals["rental_date"])
rentals["return_date"] = pd.to_datetime(rentals["return_date"])

# Calculate the time each DVD was rented
date_differences = (rentals["return_date"] - rentals["rental_date"])
rentals["rental_length_days"] = date_differences.dt.days

# Print a table of counts by the number of days a DVD was rented
rentals["rental_length_days"].value_counts(dropna= False)

7    1832
1    1829
8    1771
5    1767
6    1758
4    1757
2    1713
3    1694
9     894
0     846
Name: rental_length_days, dtype: int64

In [5]:

dict_bool = {False:0, True:1}
# Function to get a single dummie variable from the "special feature" column
get_feature_dummy = lambda col: (rentals["special_features"].str.find(col) >= 0).map(dict_bool)

# Creating important dummie columns
rentals["deleted_scenes"] = get_feature_dummy("Deleted Scenes")
rentals["behind_the_scenes"] = get_feature_dummy("Behind the Scenes")

# Observing it's behavior in counts
rentals.groupby(["deleted_scenes", "behind_the_scenes"])["special_features"].count()

deleted_scenes  behind_the_scenes
0               0                    3357
                1                    4616
1               0                    3997
                1                    3891
Name: special_features, dtype: int64

In [6]:
# Creating a list with columns names that has only numeric data types
numeric_columns = []
for i, v in zip(rentals.dtypes.index, rentals.dtypes.values):
    if v == "int64" or v == "float64":
        numeric_columns.append(i)

# Removing the variable that we want to predict
numeric_columns.remove("rental_length_days")
# Assing Independent variables in X
X = rentals[numeric_columns]
# Assing Dependent varibales in y
y = rentals["rental_length_days"]

# Spliting the data in test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 69)

# Checking data shape
(X_train.shape, y_train.shape)

((12688, 14), (12688,))

In [7]:
# Setting up GridSearch basic parameters to find a good model

parameters = {
    "n_estimators": np.arange(20, 200, 40),
    "max_depth": np.arange(2, 21, 4)
}

rfr_model = RandomForestRegressor()
gs_model = GridSearchCV(rfr_model, parameters)

# Fitting with train data
gs_model.fit(X_train, y_train)

best_model = gs_model.best_estimator_
best_model

RandomForestRegressor(max_depth=14, n_estimators=180)

In [10]:
# Predicting
y_pred = best_model.predict(X_test)

In [11]:
# Calculating metric to confirmar model avaliability
best_mse = mean_squared_error(y_test, y_pred)
best_mse

1.973098731866759

In [13]:
if best_mse <= 3:
    print("Eba!")
else:
    print("Opa...")

Eba!
